In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install fastBPE sacremoses

In [0]:
import torch
# English to German translation
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
en2de.cuda()
en2de.translate("Machine learning is great!")  # 'Maschinelles Lernen ist großartig!'




Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


'Maschinelles Lernen ist großartig!'

In [0]:
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')
de2en.cuda()
de2en.translate("Maschinelles Lernen ist großartig!")

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


'Machine learning is great!'

In [0]:
import time
start = time.time()
out = de2en.translate(en2de.translate("I am testing a randomly large sentence. I am testing a randomly large sentence. I am testing a randomly large sentence. I am testing a randomly large sentence.")) # 'Maschinelles Lernen ist großartig!'
end = time.time()
print(out)
print(end-start)

I tested a randomly large set. I tested a randomly large set. I tested a randomly large set. I tested a randomly large set.
1.7691311836242676


In [0]:
import pandas as pd
import os
import time 
file =  'Kaggle2_Mixed_a.csv'
df_input = pd.read_csv(file)
df_input.drop(['Unnamed: 0'], inplace=True,axis=1)
print(df_input.shape)
df_input.head()

(2535, 3)


,clean_text,clean_title,label
0,videos 15 civilians killed in single us airstr...,15 civilians killed in single us airstrike hav...,0
1,print \nan iranian woman has been sentenced to...,iranian woman jailed for fictional unpublished...,0
2,"in these trying times, jackie mason is the voi...",jackie mason: hollywood would love trump if he...,1
3,ever wonder how britains most iconic pop piani...,life: life of luxury: elton johns 6 favorite s...,0
4,the bbc produced spoof on the real housewives ...,"bbc comedy sketch ""real housewives of isis"" ca...",1


In [0]:
def paraphrase(input):
  return de2en.translate(en2de.translate(input))

def calculate_stats(beginning, reset, now, i, trans_count, unchanged, num):
  per = (now-beginning)/(i+1)
  mega_time = (per)*(num - i)
  print("Stats every 100 items")
  print("Total translated = {}".format(trans_count))
  print("Total unchanged = {}".format(unchanged))  
  print("Overall time elapsed = {} seconds".format(now - beginning))
  print("Overall time since last calc = {} seconds".format(now-reset))
  print("estimated time left = {} seconds".format(mega_time))
  print()


In [0]:
text = df_input['clean_text'].iloc[2]
print("Original")
print(text)
print('--------')
print("Back translated")
print(paraphrase(text))
print(df_input['label'].iloc[2])

Original
in these trying times, jackie mason is the voice of reason. [in this weeks exclusive clip for breitbart news, jackie discusses the looming threat of north korea, and explains how president donald trump could win the support of the hollywood left if the u. s. needs to strike first.  if he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do, jackie says. except the hollywood left. theyll get nauseous.  [trump] could win the left over, theyll fall in love with him in a minute. if he bombed them for a better reason, jackie explains. like if they have no transgender toilets.  jackie also says its no surprise that hollywood celebrities didnt support trumps strike on a syrian airfield this month. they were infuriated, he says. because it might only save lives. that doesnt mean anything to them. if it only saved the environment, or climate change! theyd be the happiest people in the world.  still

In [0]:
df_output = df_input.copy()
num = df_input.shape[0]
unchanged = 0
trans_count = 0
drop = []

beginning = time.time()
reset = beginning 
for i in range(num):
  og_text = df_input['clean_text'].iloc[i]
  if len(og_text) < 3501:
    try:
      translated = paraphrase(og_text)
      df_output.loc[i] = [translated,df_input.clean_title[i],df_input.label[i]]
      trans_count += 1
    except:
      unchanged += 1
      drop.append(i)
  else:
    unchanged += 1
    drop.append(i)
  if (i+1) % 50 == 0: #calculate stats every 50 
    calculate_stats(beginning, reset, time.time(), i, trans_count, unchanged, num)
    reset = time.time()


end = time.time()

Stats every 100 items
Total translated = 50
Total unchanged = 0
Overall time elapsed = 405.03657245635986 seconds
Overall time since last calc = 405.03657245635986 seconds
estimated time left = 20138.418382530213 seconds

Stats every 100 items
Total translated = 100
Total unchanged = 0
Overall time elapsed = 790.6807906627655 seconds
Overall time since last calc = 385.6432104110718 seconds
estimated time left = 19260.984060544968 seconds

Stats every 100 items
Total translated = 150
Total unchanged = 0
Overall time elapsed = 1189.959421634674 seconds
Overall time since last calc = 399.2783508300781 seconds
estimated time left = 18928.287866802213 seconds

Stats every 100 items
Total translated = 200
Total unchanged = 0
Overall time elapsed = 1575.722739458084 seconds
Overall time since last calc = 385.76303124427795 seconds
estimated time left = 18404.441596870423 seconds

Stats every 100 items
Total translated = 250
Total unchanged = 0
Overall time elapsed = 1962.249549627304 seconds


In [0]:
df_output.drop(df_output.index[drop], inplace=True)
df_output.to_csv('Kaggle2_mixed_a_bt.csv')
!cp Kaggle2_mixed_a_bt.csv "/content/drive/My Drive/"